In [11]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### FIX ME #####
# Update with your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Update with your MongoDB credentials
username = "aacuser"
password = "SNHU1234"

# Connect to the database via CRUD Module
db = AnimalShelter(username, password)

# Retrieve all documents from the collection
df = pd.DataFrame.from_records(db.read({}))

# Drop '_id' column if it exists
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

## Debug: Ensure DataFrame is valid
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Attempt to load the logo
logo_path = './Grazioso Salvare Logo.png' 
try:
    with open(logo_path, 'rb') as img_file:
        encoded_image = base64.b64encode(img_file.read()).decode()
except FileNotFoundError:
    print(f"Logo not found at '{logo_path}'.")
    encoded_image = None

# Dashboard layout
app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Img(
        src=f'data:image/png;base64,{encoded_image}' if encoded_image else None,
        style={'height': '10%', 'width': '10%'}
    ) if encoded_image else html.P("Logo not found. Please ensure the file path is correct."),
    html.Center(html.P("Dashboard developed by [Eskinder]")),
    html.Hr(),
    html.Div([
        
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water'},
                {'label': 'Mountain Rescue', 'value': 'Mountain'},
                {'label': 'Disaster Tracking', 'value': 'Disaster'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            labelStyle={'display': 'block'}
        )
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        style_table={'overflowX': 'auto'},
        page_size=10,
        sort_action='native',
        filter_action='native'
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
         style={'display': 'flex'},
         children=[
             html.Div(id='graph-id', className='col s12 m6'),
             html.Div(id='map-id', className='col s12 m6'),
         ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to filter data table based on radio button selection
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    try:
        if filter_type == 'Water':
            data = db.read({
                "breed": {"$in": ["Labrador Retriever Mix", "Newfoundland", "Chesapeake Bay Retriever"]},
                "age_upon_outcome_in_weeks": {"$lte": 156},
                "sex_upon_outcome": "Intact Female"
            })
        elif filter_type == 'Mountain':
            data = db.read({
                "breed": {"$in": ["German Shepherd", "Siberian Husky", "Rottweiler"]},
                "age_upon_outcome_in_weeks": {"$lte": 156},
                "sex_upon_outcome": "Intact Male"
            })
        elif filter_type == 'Disaster':
            data = db.read({
                "breed": {"$in": ["Bloodhound", "Rottweiler", "Golden Retriever"]},
                "age_upon_outcome_in_weeks": {"$lte": 300},
                "sex_upon_outcome": "Intact Male"
            })
        else:  # Reset or no filter
            data = db.read({})
        
        # Convert data to DataFrame and back to a list of dictionaries
        df = pd.DataFrame.from_records(data)
        if '_id' in df.columns:
            df.drop(columns=['_id'], inplace=True)  # Remove '_id' column if it exists
        return df.to_dict('records')  # Convert to Dash-compatible format
    except Exception as e:
        print(f"Error in update_dashboard: {e}")
        return []

# Callback to generate a pie chart based on the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None:
        return "No data available"
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed', title='Preferred Breeds Distribution')
    return [dcc.Graph(figure=fig)]

# Callback to highlight a cell in the data table when selected
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []  # Return an empty style list if no columns are selected
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Callback to update the geo-location chart for the selected row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or index is None:
        return "No data available"
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0
    lat, lon = dff.iloc[row, 13], dff.iloc[row, 14]
    return [
        dl.Map(style={'width': '100%', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

# Run the server
app.run_server(debug=True)


Connected to MongoDB at nv-desktop-services.apporto.com:34734, Database: AAC, Collection: animals
Dash app running on http://127.0.0.1:27756/
